In [77]:
import random

In [78]:
def get_stats(CH):
    print(f'(R,G,B,A) = ({CH[0]},{CH[1]},{CH[2]},{CH[3]})')
    print(f'Roughness Factor = {CH[4]}')
    print(f'Number of vertices = {len(CH[5])}')
    print(f'Number of faces = {len(CH[6])}')

### Importing function for generating chromosome

In [142]:
from CGen import generate_chromosome

In [143]:
%%time
L,B,H=5,5,5

P1=[]

for _ in range(10):
    CH=generate_chromosome(L,B,H,random.random())
    P1.append(CH)
    get_stats(CH)
    print('-'*125)

(R,G,B,A) = (213,105,207,0.8037697479760377)
Roughness Factor = 0.6364850462386535
Number of vertices = 56
Number of faces = 774
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (225,218,18,0.15945798987960247)
Roughness Factor = 0.5619251109581315
Number of vertices = 59
Number of faces = 904
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (253,126,76,0.021582159642815513)
Roughness Factor = 0.7721306092555844
Number of vertices = 66
Number of faces = 1079
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (212,83,159,0.9616774757585655)
Roughness Factor = 0.9315430655948329
Number of vertices = 121
Number of faces = 3713
-------------------------------------------------------------------------------------------------------

In [140]:
len(P1[0])

7

## Mutation

In [81]:
def mutate(C):
    for 

IndentationError: expected an indented block (Temp/ipykernel_28412/3924105588.py, line 2)